In [ ]:
!pip install sentence_transformers
!pip install qdrant-client
!pip install langchain
!pip install -U langchain-community

!pip install pypdf
!pip install openai

In [ ]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient

from qdrant_client import models
from qdrant_client.http.models import VectorParams, Distance

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from openai import OpenAI



In [ ]:
openai_client = OpenAI()


In [ ]:
def small_embedding(text,model="text-embedding-3-small"):
  text = text.replace("\n", " ")
  return openai_client.embeddings.create(input = [text], model=model,dimensions=128).data[0].embedding



In [ ]:
def large_embedding(text, model="text-embedding-3-large"):
  text = text.replace("\n", " ")
  return openai_client.embeddings.create(input = [text], model=model,dimensions=1024).data[0].embedding


In [ ]:
loaders = [
    PyPDFLoader("/content/TEGI0570.pdf"),
]

In [ ]:
docs = []
chunk_size=1050
chunk_overlap = 50


In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    )

In [ ]:
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
splits = r_splitter.split_documents(docs)


In [ ]:
len(splits)

309

In [ ]:
client = QdrantClient(":memory:")
COLLECTION_NAME = "multi_stage_db"


In [ ]:
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "small-embedding": models.VectorParams(
            size=128,
            distance=models.Distance.COSINE,
            datatype=models.Datatype.FLOAT16
        ),

        "large-embedding": models.VectorParams(
            size=1024,
            distance=models.Distance.COSINE,
            datatype=models.Datatype.FLOAT16
        ),

    },

)

<ipython-input-64-0cdb92f01c42>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:

for i in range(0,len(splits)):
  client.upsert(
      collection_name=COLLECTION_NAME,
      points=[
          models.PointStruct(
              id=i,
              vector={
                  "small-embedding":small_embedding(splits[i].page_content),
                  "large-embedding":large_embedding(splits[i].page_content),

              },


          )
      ],
  )


In [ ]:
query_text = "what are  measurement and Mismeasurement of Risk "

small_vector = small_embedding(query_text)
large_vector = large_embedding(query_text)

In [ ]:


result = client.query_points(
    collection_name= COLLECTION_NAME,
    prefetch=models.Prefetch(
        query=small_vector,
        using="small-embedding",
        limit=50,
    ),
    query=large_vector,
    using="large-embedding",
    limit=5,
)



In [ ]:
result

QueryResponse(points=[ScoredPoint(id=48, version=0, score=0.5731798965683417, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=49, version=0, score=0.511801688120779, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=53, version=0, score=0.5016154467017568, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=50, version=0, score=0.5012954712589167, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=51, version=0, score=0.4720819299720585, payload={}, vector=None, shard_key=None, order_value=None)])

In [ ]:

ids = [item.id for item in result.points]

for i in ids:
  print(splits[i].page_content)
  print('\n')
  print('-'*75)

10 
 Trend Reversion Yield Growth Quality
Short Term
Medium Term
Long TermIntrinsic
RelativeLiquid
IliquidPhenomenon
Time Horizon Bet Structure Instruments
High
Frequency
ImplementationPrice FundamentalAlpha
InputReturn 
Category
 
Figure 2.3.4– Taxonomy of Theory-Driven Models 
 
2.3.2. Risk Models 
An attentive trader should not worry only about the avoidance of risk or reduction of loss. In addition 
to these goals, the intentional selection and sizing of exposures to improve the quality and 
consistency of returns are most essential for its survival in the financial markets. Risk exposures will 
most likely not produce profits over the long run; however, they can seriously undermine the returns 
of a strategy over time. Furthermore, any attempt to forecast these risk exposures should be avoided 
simply because they cannot be predicted successfully. What matters is the ability to understand and 
measure several exposures and to be intentional about the selection of such exposures (N